In [ ]:
# | default_exp cli.model

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import os
import typer

from typer import echo
from tabulate import tabulate
import pandas as pd

from airt.cli import helper
from airt.logger import get_logger, set_level

In [ ]:
from contextlib import contextmanager
import logging

import pytest
from typer.testing import CliRunner
from datetime import timedelta

import airt.sanitizer
from airt.client import Client, DataBlob

from airt.constant import SERVICE_USERNAME, SERVICE_PASSWORD, SERVICE_TOKEN

In [ ]:
# | exporti

app = typer.Typer(
    help="A set of commands for querying the model training, evaluation, and prediction status."
)

In [ ]:
runner = CliRunner()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
set_level(logging.WARNING)

In [ ]:
# Testing logger settings

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
_airt_service_token = None


@contextmanager
def set_airt_service_token_envvar():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with set_airt_service_token_envvar():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
TEST_S3_URI = "s3://test-airt-service/ecommerce_behavior_notebooks"
RANDOM_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
_model = None


@contextmanager
def generate_model(force_create: bool = False):
    global _model

    if _model is None or force_create:

        db = DataBlob.from_s3(
            uri=TEST_S3_URI,
            access_key=os.environ["AWS_ACCESS_KEY_ID"],
            secret_key=os.environ["AWS_SECRET_ACCESS_KEY"],
        )

        db.progress_bar()

        ds = db.to_datasource(
            file_type="parquet", index_column="user_id", sort_by="event_time"
        )

        display(f"{ds.uuid=}")

        ds.progress_bar()

        _model = ds.train(
            client_column="user_id",
            target_column="category_code",
            target="*purchase",
            predict_after=timedelta(hours=3),
        )

        _model.wait()

    yield _model

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def ls(
    offset: int = typer.Option(
        0,
        "--offset",
        "-o",
        help="The number of models to offset at the beginning. If None, then the default value **0** will be used.",
    ),
    limit: int = typer.Option(
        100,
        "--limit",
        "-l",
        help="The maximum number of models to return from the server. If None, "
        "then the default value **100** will be used.",
    ),
    disabled: bool = typer.Option(
        False,
        "--disabled",
        help="If set to **True**, then only the deleted models will be returned. Else, the default value "
        "**False** will be used to return only the list of active models.",
    ),
    completed: bool = typer.Option(
        False,
        "--completed",
        help="If set to **True**, then only the models that are successfully downloaded "
        "to the server will be returned. Else, the default value **False** will be used to "
        "return all the models.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only uuids of model separated by space",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the list of models."""

    from airt.client import Model

    mx = Model.ls(offset=offset, limit=limit, disabled=disabled, completed=completed)

    df = Model.as_df(mx)

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df, "quite_column_name": "model_uuid"}

In [ ]:
def assert_has_help(xs: List[str]):
    result = runner.invoke(app, xs + ["--help"])

    display(result.stdout)
    assert " ".join(xs) in result.stdout

In [ ]:
assert_has_help(["ls"])

'Usage: ls [OPTIONS]\n\n  Return the list of models.\n\nOptions:\n  -o, --offset INTEGER            The number of models to offset at the\n                                  beginning. If None, then the default value\n                                  **0** will be used.  [default: 0]\n  -l, --limit INTEGER             The maximum number of models to return from\n                                  the server. If None, then the default value\n                                  **100** will be used.  [default: 100]\n  --disabled                      If set to **True**, then only the deleted\n                                  models will be returned. Else, the default\n                                  value **False** will be used to return only\n                                  the list of active models.\n  --completed                     If set to **True**, then only the models that\n                                  are successfully downloaded to the server will\n                        

In [ ]:
# Tests for model_ls
# Testing positive scenario. Saving the token in env variable


def get_ids_from_result(result) -> List[int]:
    return [uuid for uuid in result.stdout[:-1].split("\n")]


with set_airt_service_token_envvar():
    with generate_model() as model:

        # Without quiet
        result = runner.invoke(app)
        display(result.stdout)

        assert "ready" in result.stdout
        assert result.exit_code == 0

        # With format
        format_str = "{'model_uuid': '{}', 'created': '{}'}"
        result = runner.invoke(app, ["--format", format_str])
        display(result.stdout)

        assert result.exit_code == 0

        # With quiet
        result = runner.invoke(app, ["-q"])
        display(result.stdout)

        assert result.exit_code == 0
        ids = get_ids_from_result(result)
        display(f"{ids=}")

100%|██████████| 1/1 [00:15<00:00, 15.19s/it]


"ds.uuid='a2a63f8a-06f1-49e4-bb22-15979ecaf59a'"

100%|██████████| 1/1 [00:30<00:00, 30.37s/it]


'model_uuid                            created         ready\nb3b17f4c-2f00-4f51-9841-5dac52bfea61  2 hours ago     True\n4b4b3909-671f-43f9-b868-d7d70a790e39  2 hours ago     True\n5e80898d-a2a0-4341-8412-157086638d43  50 minutes ago  True\n5c9b02a4-23a0-4743-b163-ad74f217d685  28 minutes ago  True\n33dfede5-9913-4cf0-8029-e3ed1972a7da  23 minutes ago  True\n38b96e82-43a4-4d6c-9275-efde8f820815  23 minutes ago  True\ndc193688-c4a1-4f2c-9a2b-5016859f66a8  21 minutes ago  True\ndab7942a-f564-4668-bb95-12058fdd35cd  19 minutes ago  True\naa2955d2-e785-4e2b-a302-2ca185ce2ece  19 minutes ago  True\n7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7  18 minutes ago  True\n4b781be8-068b-4eb2-bf35-abaf5920d9b4  17 minutes ago  True\n9d6514fd-b190-44cb-b134-5e2ef0117898  16 minutes ago  True\n1760dfd2-a718-4323-8498-defe1125c93d  7 minutes ago   True\nb77ba7d2-f0a1-4ccf-aa31-2fa1d124adba  2 minutes ago   True\n2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4  now             True\n'

'model_uuid                            created\nb3b17f4c-2f00-4f51-9841-5dac52bfea61  2 hours ago\n4b4b3909-671f-43f9-b868-d7d70a790e39  2 hours ago\n5e80898d-a2a0-4341-8412-157086638d43  50 minutes ago\n5c9b02a4-23a0-4743-b163-ad74f217d685  28 minutes ago\n33dfede5-9913-4cf0-8029-e3ed1972a7da  23 minutes ago\n38b96e82-43a4-4d6c-9275-efde8f820815  23 minutes ago\ndc193688-c4a1-4f2c-9a2b-5016859f66a8  21 minutes ago\ndab7942a-f564-4668-bb95-12058fdd35cd  19 minutes ago\naa2955d2-e785-4e2b-a302-2ca185ce2ece  19 minutes ago\n7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7  18 minutes ago\n4b781be8-068b-4eb2-bf35-abaf5920d9b4  17 minutes ago\n9d6514fd-b190-44cb-b134-5e2ef0117898  16 minutes ago\n1760dfd2-a718-4323-8498-defe1125c93d  7 minutes ago\nb77ba7d2-f0a1-4ccf-aa31-2fa1d124adba  2 minutes ago\n2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4  now\n'

'b3b17f4c-2f00-4f51-9841-5dac52bfea61\n4b4b3909-671f-43f9-b868-d7d70a790e39\n5e80898d-a2a0-4341-8412-157086638d43\n5c9b02a4-23a0-4743-b163-ad74f217d685\n33dfede5-9913-4cf0-8029-e3ed1972a7da\n38b96e82-43a4-4d6c-9275-efde8f820815\ndc193688-c4a1-4f2c-9a2b-5016859f66a8\ndab7942a-f564-4668-bb95-12058fdd35cd\naa2955d2-e785-4e2b-a302-2ca185ce2ece\n7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7\n4b781be8-068b-4eb2-bf35-abaf5920d9b4\n9d6514fd-b190-44cb-b134-5e2ef0117898\n1760dfd2-a718-4323-8498-defe1125c93d\nb77ba7d2-f0a1-4ccf-aa31-2fa1d124adba\n2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4\n'

"ids=['b3b17f4c-2f00-4f51-9841-5dac52bfea61', '4b4b3909-671f-43f9-b868-d7d70a790e39', '5e80898d-a2a0-4341-8412-157086638d43', '5c9b02a4-23a0-4743-b163-ad74f217d685', '33dfede5-9913-4cf0-8029-e3ed1972a7da', '38b96e82-43a4-4d6c-9275-efde8f820815', 'dc193688-c4a1-4f2c-9a2b-5016859f66a8', 'dab7942a-f564-4668-bb95-12058fdd35cd', 'aa2955d2-e785-4e2b-a302-2ca185ce2ece', '7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7', '4b781be8-068b-4eb2-bf35-abaf5920d9b4', '9d6514fd-b190-44cb-b134-5e2ef0117898', '1760dfd2-a718-4323-8498-defe1125c93d', 'b77ba7d2-f0a1-4ccf-aa31-2fa1d124adba', '2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4']"

In [ ]:
# Tests for model_ls
# Testing positive scenario.
# Testing by passing different values for  limit


def get_ids_from_result(result) -> List[int]:
    return [uuid for uuid in result.stdout[:-1].split("\n")]


with set_airt_service_token_envvar():
    with generate_model() as model:
        for limit in [1, 10, 1000]:
            offset = 1
            result = runner.invoke(app, ["--offset", offset, "--limit", limit, "-q"])

            assert result.exit_code == 0

            ids = get_ids_from_result(result)
            display(f"{ids=}")
            assert limit >= len(ids) >= 0

"ids=['4b4b3909-671f-43f9-b868-d7d70a790e39']"

"ids=['4b4b3909-671f-43f9-b868-d7d70a790e39', '5e80898d-a2a0-4341-8412-157086638d43', '5c9b02a4-23a0-4743-b163-ad74f217d685', '33dfede5-9913-4cf0-8029-e3ed1972a7da', '38b96e82-43a4-4d6c-9275-efde8f820815', 'dc193688-c4a1-4f2c-9a2b-5016859f66a8', 'dab7942a-f564-4668-bb95-12058fdd35cd', 'aa2955d2-e785-4e2b-a302-2ca185ce2ece', '7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7', '4b781be8-068b-4eb2-bf35-abaf5920d9b4']"

"ids=['4b4b3909-671f-43f9-b868-d7d70a790e39', '5e80898d-a2a0-4341-8412-157086638d43', '5c9b02a4-23a0-4743-b163-ad74f217d685', '33dfede5-9913-4cf0-8029-e3ed1972a7da', '38b96e82-43a4-4d6c-9275-efde8f820815', 'dc193688-c4a1-4f2c-9a2b-5016859f66a8', 'dab7942a-f564-4668-bb95-12058fdd35cd', 'aa2955d2-e785-4e2b-a302-2ca185ce2ece', '7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7', '4b781be8-068b-4eb2-bf35-abaf5920d9b4', '9d6514fd-b190-44cb-b134-5e2ef0117898', '1760dfd2-a718-4323-8498-defe1125c93d', 'b77ba7d2-f0a1-4ccf-aa31-2fa1d124adba', '2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4']"

In [ ]:
# Tests for model_ls
# Testing positive scenario.
# Testing by passing large value for offset.

with set_airt_service_token_envvar():
    with generate_model() as model:
        limit = 10
        offset = 1_000_000
        result = runner.invoke(app, ["--offset", offset, "--limit", limit])

        assert result.exit_code == 0

        display(result.stdout)

'model_uuid    created    ready\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def details(
    uuid: str = typer.Argument(
        ...,
        help="Model uuid",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the details of a model."""

    from airt.client import Model

    model = Model(uuid=uuid)
    df = model.details()

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df, "quite_column_name": "model_uuid"}

In [ ]:
assert_has_help(["details"])

'Usage: root details [OPTIONS] UUID\n\n  Return the details of a model.\n\nArguments:\n  UUID  Model uuid  [required]\n\nOptions:\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for details
# Testing positive scenario

# Helper function to extract ID


def extract_id(res) -> str:
    r = (res.split("\n")[1]).strip()
    return r.split(" ")[0]


with set_airt_service_token_envvar():
    with generate_model() as model:

        # Getting Details of the model
        format_str = "{'model_uuid': '{}', 'created': '{}'}"
        result = runner.invoke(app, ["details", model.uuid, "--format", format_str])

        result_id = extract_id(result.stdout)

        display(result.stdout)

        assert result.exit_code == 0
        assert result_id == model.uuid

'model_uuid                            created\n2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4  now\n'

In [ ]:
# Tests for details
# Testing negative scenario. Passing invalie id

with set_airt_service_token_envvar():

    result = runner.invoke(app, ["details", RANDOM_UUID_FOR_TESTING])

    display(result.stdout)

    assert result.exit_code == 1

'Error: The model uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def rm(
    uuid: str = typer.Argument(
        ...,
        help="Model uuid",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output the deleted Model uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Delete a model from the server."""

    from airt.client import Model

    model = Model(uuid=uuid)
    df = model.delete()

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df, "quite_column_name": "model_uuid"}

In [ ]:
assert_has_help(["rm"])

'Usage: root rm [OPTIONS] UUID\n\n  Delete a model from the server.\n\nArguments:\n  UUID  Model uuid  [required]\n\nOptions:\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -q, --quiet        Output the deleted Model uuid only.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# Tests for model rm
# Testing positive scenario with quite

with set_airt_service_token_envvar():
    with generate_model() as model:

        # Deleting the created model from the server
        result = runner.invoke(app, ["rm", model.uuid, "-q"])
        deleted_uuid = result.stdout[:-1]

        display(deleted_uuid)

        assert result.exit_code == 0
        assert deleted_uuid == model.uuid

        # List the existing model ids in server and make sure the deleted id is not present in the server
        ls_result = runner.invoke(app, ["ls", "-q"])
        ls_ids = get_ids_from_result(ls_result)

        display(ls_ids)
        assert deleted_uuid not in ls_ids

        # Testing negative scenario. Deleting already deleted model
        format_str = "{'model_uuid': '{}'}"
        result = runner.invoke(app, ["rm", deleted_uuid, "-f", format_str])
        display(result.stdout)
        assert result.exit_code == 1

        # Testing negative scenario. Getting the details of the deleted model
        result = runner.invoke(app, ["details", deleted_uuid])
        display(result.stdout)
        assert result.exit_code == 1

'2c8757da-0a75-4dfe-b1e9-952dbcfbb6b4'

['b3b17f4c-2f00-4f51-9841-5dac52bfea61',
 '4b4b3909-671f-43f9-b868-d7d70a790e39',
 '5e80898d-a2a0-4341-8412-157086638d43',
 '5c9b02a4-23a0-4743-b163-ad74f217d685',
 '33dfede5-9913-4cf0-8029-e3ed1972a7da',
 '38b96e82-43a4-4d6c-9275-efde8f820815',
 'dc193688-c4a1-4f2c-9a2b-5016859f66a8',
 'dab7942a-f564-4668-bb95-12058fdd35cd',
 'aa2955d2-e785-4e2b-a302-2ca185ce2ece',
 '7f6b2c6b-b4c9-4f33-b781-d4bba9d67db7',
 '4b781be8-068b-4eb2-bf35-abaf5920d9b4',
 '9d6514fd-b190-44cb-b134-5e2ef0117898',
 '1760dfd2-a718-4323-8498-defe1125c93d',
 'b77ba7d2-f0a1-4ccf-aa31-2fa1d124adba']

'Error: The model has already been deleted.\n'

'Error: The model has already been deleted.\n'

In [ ]:
# Tests for model rm
# Testing negative scenario. Deleting invalid data source

with set_airt_service_token_envvar():
    with generate_model() as model:

        result = runner.invoke(app, ["rm", RANDOM_UUID_FOR_TESTING, "-q"])

        display(result.stdout)

'Error: The model uuid is incorrect. Please try again.\n'

In [ ]:
# | exporti


@app.command()
@helper.requires_auth_token
def predict(
    data_uuid: str = typer.Option(
        ...,
        "--data_uuid",
        help="DataSource uuid.",
    ),
    uuid: str = typer.Option(
        ...,
        "--uuid",
        help="Model uuid.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output the prediction id only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Run predictions against the trained model."""

    from airt.client import Model

    model = Model(uuid=uuid)

    prediction = model.predict(data_uuid=data_uuid)

    if quiet:
        prediction.wait()

        typer.echo(f"{prediction.uuid}")
    else:
        typer.echo(f"Running predictions for prediction uuid: {prediction.uuid}")
        prediction.progress_bar()

In [ ]:
assert_has_help(["predict"])

'Usage: root predict [OPTIONS]\n\n  Run predictions against the trained model.\n\nOptions:\n  --data_uuid TEXT  DataSource uuid.  [required]\n  --uuid TEXT       Model uuid.  [required]\n  -q, --quiet       Output the prediction id only.\n  -d, --debug       Set logger level to DEBUG and output everything.\n  --help            Show this message and exit.\n'

In [ ]:
# Tests for model predict
# Testing positive scenario without quite

with set_airt_service_token_envvar():
    with generate_model(force_create=True) as model:
        # Running prediction
        data_uuid = model.details()["datasource_uuid"][0]
        result = runner.invoke(
            app, ["predict", "--uuid", model.uuid, "--data_uuid", data_uuid]
        )

        display(result.stdout)

        assert result.exit_code == 0
        assert "Running predictions for prediction uuid" in result.stdout

100%|██████████| 1/1 [00:15<00:00, 15.15s/it]


"ds.uuid='e9103626-e10d-4e54-9965-ed04400b21c7'"

100%|██████████| 1/1 [00:30<00:00, 30.31s/it]


'Running predictions for prediction uuid: 9dbb3261-bba7-47df-bd57-21358c10ff79\n\r  0%|          | 0/3 [00:00<?, ?it/s]\r  0%|          | 0/3 [00:05<?, ?it/s]\r100%|██████████| 3/3 [00:10<00:00,  1.68s/it]\r100%|██████████| 3/3 [00:10<00:00,  3.38s/it]\n'

In [ ]:
# Tests for model predict
# Testing positive scenario without quite

with set_airt_service_token_envvar():
    with generate_model() as model:
        data_uuid = model.details()["datasource_uuid"][0]
        result = runner.invoke(
            app, ["predict", "--uuid", model.uuid, "--data_uuid", data_uuid, "-q"]
        )

        display(result.stdout)

        assert result.exit_code == 0
        assert len(result.stdout.replace("-", "").replace("\n", "")) == 32

'b213e99d-6215-44a9-a658-0ce6fdab045c\n'

In [ ]:
# | exporti


@app.command()
@helper.requires_auth_token
def evaluate(
    uuid: str = typer.Argument(
        ...,
        help="Model uuid.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> None:
    """Return the evaluation metrics of the trained model.

    Currently, this command returns the model's accuracy, precision, and recall. In the future, more performance metrics will be added.
    """

    from airt.client import Model

    model = Model(uuid=uuid)

    df = model.evaluate()

    typer.echo(tabulate(df, headers="keys", tablefmt="plain"))  # type: ignore

In [ ]:
assert_has_help(["evaluate"])

"Usage: root evaluate [OPTIONS] UUID\n\n  Return the evaluation metrics of the trained model.\n\n  Currently, this command returns the model's accuracy, precision, and recall.\n  In the future, more performance metrics will be added.\n\nArguments:\n  UUID  Model uuid.  [required]\n\nOptions:\n  -d, --debug  Set logger level to DEBUG and output everything.\n  --help       Show this message and exit.\n"

In [ ]:
# Tests for evaluate
# Testing positive scenario


with set_airt_service_token_envvar():
    with generate_model() as model:
        # Getting Details of the model
        result = runner.invoke(app, ["evaluate", model.uuid])

        display(result.stdout)

        assert result.exit_code == 0
        assert "eval" in result.stdout
        assert "accuracy" in result.stdout

'             eval\naccuracy    0.985\nrecall      0.962\nprecision   0.934\n'

In [ ]:
# Tests for evaluate
# Testing negative scenario, passing wrong id


with set_airt_service_token_envvar():
    with generate_model() as model:
        result = runner.invoke(app, ["evaluate", RANDOM_UUID_FOR_TESTING])

        display(result.stdout)

        assert result.exit_code == 1

'Error: The model uuid is incorrect. Please try again.\n'